# Final Project
### Introduction to Computational Medicine: Imaging 
### Group: 6

In [104]:
# import statements
import pandas as pd
import scipy.stats as stats

## Part 1: Computing patient "bar codes"

First we load our dataset. For our project, we have selected Alzheimer's Disease, so we will be using the ADNI dataset.

In [88]:
# load ADNI dataset into a pandas dataframe
working_directory = "C:/Users/lizan/Desktop/Classes/ICM Imaging/ICM_ImagingFinalProject_Group6/"
dataset = "ADNI_ICM_random200.xlsx"
path = working_directory + dataset
ADNI_dataset = pd.read_excel(path)

# look at our dataset format
ADNI_dataset.head()

'Subject_ID' 'Gender'    'Age' 'Diagnosis'  'ACR_R'  'AGWM_R'  'AG_R'  \
0  'Subject_1'      'M'  69.6712    'AD/MCI'    14005      2932   10758   
1  'Subject_2'      'F'  85.3836    'AD/MCI'     8811      2115    9418   
2  'Subject_3'      'F'  86.2904    'NORMAL'    11982      2634   11015   
3  'Subject_4'      'M'  73.9342    'NORMAL'    11919      3688   11322   
4  'Subject_5'      'M'  79.9918    'AD/MCI'    10588      3010   11657   

   'ALIC_R'  'Amyg_R'  'AnteriorBasalForebrain_R'  ...  'TempSul_L'  \
0      2073      1725                         363  ...         1349   
1      2317      1590                         276  ...         1368   
2      2589      1803                         299  ...         1337   
3      2058      1868                         217  ...         1187   
4      2161      1275                         232  ...         2131   

   'Thalamus_L'  'dorsalWM_ACC_L'  'dorsal_ACC_L'  'rostralWM_ACC_L'  \
0          5452              1424           10839                 12   
1          4578              1187            9102                  5   
2          5987              1601            9902                 92   
3          5005              1705           12301                 12   
4          5239              1517           11344                 45   

   'rostral_ACC_L'  'subcallosalWM_ACC_L'  'subcallosal_ACC_L'  \
0             2082                     95                  369   
1             1782                     38                  342   
2             1956                     39                  472   
3             1724                     97                  329   
4             1823                     28                  312   

   'subgenualWM_ACC_L'  'subgenual_ACC_L'  
0                  226               1828  
1                  128               1438  
2                  127               1644  
3                  133               1485  
4                   90               1758  

[5 rows x 278 columns]

Next we will clean up the dataset. For this project, we are only using the subject ID, diagnosis, and structures within the chosen ROI. Since we chose the limbic system as our ROI, the structures we will be examining are the amygdala, fimbria, hippocampus, and mammillary bodies. 

In [89]:
# get relevant columns
structure_list = ["Amyg_L", "Amyg_R", 
                  "Fimbria_L", "Fimbria_R", 
                  "Hippo_L", "Hippo_R", 
                  "Mammillary_L", "Mammillary_R"]
pt_info = ["Subject_ID", "Diagnosis"]
relevant_columns = pt_info + structure_list

# remove all ' symbols from dataset  
ADNI_dataset.columns = ADNI_dataset.columns.str.replace('[\']', '', regex=True)
ADNI_dataset = ADNI_dataset.replace('[\']', '', regex=True)

# make new cleaned dataframe with only relevant columns
ADNI_dataset_cleaned = ADNI_dataset[relevant_columns]

# make subject_id only the number of the subject for clarity
ADNI_dataset_cleaned = ADNI_dataset_cleaned.replace({'Subject_':''}, regex=True)
ADNI_dataset_cleaned['Subject_ID'] = ADNI_dataset_cleaned['Subject_ID'].astype(int)

# change string in diagnosis to binary
ADNI_dataset_final = ADNI_dataset_cleaned.replace({'AD/MCI': 1, 'NORMAL': 0}, regex=True)

ADNI_dataset_final.head()

Subject_ID  Diagnosis  Amyg_L  Amyg_R  Fimbria_L  Fimbria_R  Hippo_L  \
0           1          1    1379    1725          0          8     2776   
1           2          1    1489    1590          0          0     2241   
2           3          0    1697    1803          0          5     3995   
3           4          0    1515    1868         40          4     3826   
4           5          1    1131    1275          0          0     2870   

   Hippo_R  Mammillary_L  Mammillary_R  
0     3041            98            74  
1     2685            45            50  
2     3875           101           101  
3     3684           118            97  
4     2549            98            75

Now we create a dataframe containing the volume means and standard deviations for the healthy subjects.

In [92]:
# subset all the healthy patients
ADNI_healthy = ADNI_dataset_final.loc[ADNI_dataset_final['Diagnosis'] == 0]

# create a dataframe for means and std dev of each structure
ADNI_describe = ADNI_healthy.iloc[:,2:].describe().iloc[1:3,:]
ADNI_describe.head()

Amyg_L       Amyg_R  Fimbria_L  Fimbria_R      Hippo_L  \
mean  1614.968750  1877.546875   7.687500  15.562500  3481.984375   
std    330.041891   438.832323   8.581773  11.180872   549.324217   

          Hippo_R  Mammillary_L  Mammillary_R  
mean  3573.609375     93.421875     79.593750  
std    686.001148     15.983925     13.558479

Define a function that calculates the zscore for a column using mean and stddev from another dataframe

In [96]:
# Function to calculate zscores for a column of values
# Parameters: 
#    mean:    double
#    std:     double 
#    values:  list(doubles)
# Returns:
#    zscores: list(doubles)

def calculate_zscore(mean, std, values):
    zscore = [(x - mean)/std for x in values]
    return zscore

Create a dataframe containing zscores corresponding to each patient

In [98]:
# Create empty dataframe with column names
ADNI_zscores = pd.DataFrame(columns = ADNI_describe.columns)
ADNI_zscores.head()

Empty DataFrame
Columns: [Amyg_L, Amyg_R, Fimbria_L, Fimbria_R, Hippo_L, Hippo_R, Mammillary_L, Mammillary_R]
Index: []

In [99]:
# Loop through to add zscores
for i in range(8):
    mean = ADNI_describe.iloc[0, i]
    std = ADNI_describe.iloc[1, i]
    ADNI_zscores.iloc[:,i] = calculate_zscore(mean, std, ADNI_dataset_final.iloc[:,i+2])

ADNI_zscores.head()

Amyg_L    Amyg_R  Fimbria_L  Fimbria_R   Hippo_L   Hippo_R  Mammillary_L  \
0 -0.714966 -0.347620  -0.895794  -0.676378 -1.285187 -0.776397      0.286421   
1 -0.381675 -0.655255  -0.895794  -1.391886 -2.259111 -1.295347     -3.029411   
2  0.248548 -0.169876  -0.895794  -0.944694  0.933903  0.439344      0.474109   
3 -0.302897 -0.021755   3.765248  -1.034132  0.626252  0.160919      1.537678   
4 -1.466386 -1.373069  -0.895794  -1.391886 -1.114068 -1.493597      0.286421   

   Mammillary_R  
0     -0.412565  
1     -2.182675  
2      1.578809  
3      1.283791  
4     -0.338810

In [100]:
# append subject ids to our dataset
ADNI_zscores = pd.concat([ADNI_dataset_final.iloc[:,0:1], ADNI_zscores], axis=1)
ADNI_zscores.head()

Subject_ID    Amyg_L    Amyg_R  Fimbria_L  Fimbria_R   Hippo_L   Hippo_R  \
0           1 -0.714966 -0.347620  -0.895794  -0.676378 -1.285187 -0.776397   
1           2 -0.381675 -0.655255  -0.895794  -1.391886 -2.259111 -1.295347   
2           3  0.248548 -0.169876  -0.895794  -0.944694  0.933903  0.439344   
3           4 -0.302897 -0.021755   3.765248  -1.034132  0.626252  0.160919   
4           5 -1.466386 -1.373069  -0.895794  -1.391886 -1.114068 -1.493597   

   Mammillary_L  Mammillary_R  
0      0.286421     -0.412565  
1     -3.029411     -2.182675  
2      0.474109      1.578809  
3      1.537678      1.283791  
4      0.286421     -0.338810

In [101]:
# reshape dataset so it can be plotted 
ADNI_zscores_melt = ADNI_zscores.melt(id_vars=["Subject_ID"], 
                                    var_name="Structure", 
                                    value_name="zscore")
ADNI_zscores_pivot = ADNI_zscores_melt.pivot("Subject_ID", "Structure", "zscore")
ADNI_zscores_pivot.head()

Structure     Amyg_L    Amyg_R  Fimbria_L  Fimbria_R   Hippo_L   Hippo_R  \
Subject_ID                                                                 
1          -0.714966 -0.347620  -0.895794  -0.676378 -1.285187 -0.776397   
2          -0.381675 -0.655255  -0.895794  -1.391886 -2.259111 -1.295347   
3           0.248548 -0.169876  -0.895794  -0.944694  0.933903  0.439344   
4          -0.302897 -0.021755   3.765248  -1.034132  0.626252  0.160919   
5          -1.466386 -1.373069  -0.895794  -1.391886 -1.114068 -1.493597   

Structure   Mammillary_L  Mammillary_R  
Subject_ID                              
1               0.286421     -0.412565  
2              -3.029411     -2.182675  
3               0.474109      1.578809  
4               1.537678      1.283791  
5               0.286421     -0.338810

In [102]:
# save data
filename = "zscores_data.xlsx"
path = working_directory + filename
ADNI_zscores.to_excel(path)

## Part 2: Comparing structure volumes between the "control" and "disease" groups

We will perform an independent t-test

In [117]:
# Create empty dataframe with column and row names
#row_names = ['statistic', 'pvalue']
ADNI_stattest = pd.DataFrame(columns = ADNI_describe.columns)

# Loop through to add statistic and p-value
for i in range(8):
    # temporary dataframe that contains diagnosis and current structure columns
    temp_df = ADNI_dataset_final.iloc[:, [1, i+2]]
    # compute t-value (statistic)
    statistic = stats.ttest_ind(temp_df.loc[temp_df['Diagnosis']==0].iloc[:,1],
                                temp_df.loc[temp_df['Diagnosis']==1].iloc[:,1]).statistic
    # compute pvalue 
    pvalue = stats.ttest_ind(temp_df.loc[temp_df['Diagnosis']==0].iloc[:,1],
                             temp_df.loc[temp_df['Diagnosis']==1].iloc[:,1]).pvalue
    # add value to 
    ADNI_stattest.iloc[:,i] = [statistic, pvalue]

ADNI_stattest = ADNI_stattest.rename(index={0: "statistic", 1: "pvalue"})
ADNI_stattest.head()

Amyg_L    Amyg_R  Fimbria_L  Fimbria_R   Hippo_L   Hippo_R  \
statistic  4.757290  4.910386   4.922597   4.670302  3.331728  3.310068   
pvalue     0.000004  0.000002   0.000002   0.000006  0.001030  0.001108   

           Mammillary_L  Mammillary_R  
statistic      0.627826     -0.741586  
pvalue         0.530841      0.459217